In [1]:
print("hello")

hello


In [2]:
%pwd

'c:\\Users\\HP VICTUS\\Downloads\\final ai tour guide\\notebooks'

In [3]:
import os
os.chdir('../')

In [4]:
%pwd


'c:\\Users\\HP VICTUS\\Downloads\\final ai tour guide'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
# Extract text from PDF files
def load_pdf_files(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )

    documents = loader.load()
    return documents

In [6]:
extracted_data = load_pdf_files("data")


In [7]:
extracted_data

[Document(metadata={'producer': 'Microsoft® Word 2021', 'creator': 'Microsoft® Word 2021', 'creationdate': '2025-11-07T17:00:24+05:30', 'author': 'Swapnil Banerjee', 'moddate': '2025-11-07T17:00:24+05:30', 'source': 'data\\Tourism Of West Bengal_1.pdf', 'total_pages': 40, 'page': 0, 'page_label': '1'}, page_content='Tourism Of West Bengal \n \nTop Tourist & Cultural Attractions in Kolkata (The City of Joy) \n \nKolkata, once the capital of British India, is a city where colonial heritage, spiritual traditions, intellectual vibrancy,  \nand Bengali culture blend seamlessly. Below is a curated list of its most famous landmarks — with short historical \ncontext and what makes them culturally special. \n \n \nVictoria Memorial \n \nBuilt: 1906–1921 \nArchitectural Style: Indo-Saracenic (mix of British, Mughal, and Venetian styles) \nHistorical Background: Constructed to honor Queen Victoria after her death, the memorial was designed by Sir \nWilliam Emerson and funded partly by Indian prin

In [8]:
len(extracted_data)


40

In [9]:
from typing import List
from langchain.schema import Document


In [11]:
def filter_to_minimal_docs(docs: List[Document]) -> List[Document]:
    """
    Given a list of Document objects, return a new list of Document objects
    containing only 'source' in metadata and the original page_content.
    """
    minimal_docs: List[Document] = []
    for doc in docs:
        src = doc.metadata.get("source")
        minimal_docs.append(
            Document(
                page_content=doc.page_content,
                metadata={"source": src}
            )
        )
    return minimal_docs

In [12]:
minimal_docs = filter_to_minimal_docs(extracted_data)

In [13]:
# Split the documents into smaller chunks
def text_split(minimal_docs):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
    )
    texts_chunk = text_splitter.split_documents(minimal_docs)
    return texts_chunk


In [14]:
texts_chunk = text_split(minimal_docs)
print(f"Number of chunks: {len(texts_chunk)}")


Number of chunks: 157


In [15]:
texts_chunk

[Document(metadata={'source': 'data\\Tourism Of West Bengal_1.pdf'}, page_content='Tourism Of West Bengal \n \nTop Tourist & Cultural Attractions in Kolkata (The City of Joy) \n \nKolkata, once the capital of British India, is a city where colonial heritage, spiritual traditions, intellectual vibrancy,  \nand Bengali culture blend seamlessly. Below is a curated list of its most famous landmarks — with short historical \ncontext and what makes them culturally special. \n \n \nVictoria Memorial \n \nBuilt: 1906–1921'),
 Document(metadata={'source': 'data\\Tourism Of West Bengal_1.pdf'}, page_content='Built: 1906–1921 \nArchitectural Style: Indo-Saracenic (mix of British, Mughal, and Venetian styles) \nHistorical Background: Constructed to honor Queen Victoria after her death, the memorial was designed by Sir \nWilliam Emerson and funded partly by Indian princes and British citizens. \nCultural Value: It symbolizes the grandeur of the British Raj era in India. Today it serves as a museum 

In [16]:
from langchain_huggingface import HuggingFaceEmbeddings

def download_embeddings():
    """
    Download and return the HuggingFace embeddings model.
    """
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(
        model_name=model_name
    )
    return embeddings



In [17]:
embedding = download_embeddings()
embedding


c:\Users\HP VICTUS\Downloads\final ai tour guide\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [18]:
vector = embedding.embed_query("Hello world")
vector


[-0.03447727486491203,
 0.03102312609553337,
 0.006734980270266533,
 0.026108933612704277,
 -0.03936205804347992,
 -0.16030246019363403,
 0.06692394614219666,
 -0.006441438104957342,
 -0.047450482845306396,
 0.014758863486349583,
 0.07087534666061401,
 0.05552757531404495,
 0.019193356856703758,
 -0.02625126577913761,
 -0.01010954286903143,
 -0.026940442621707916,
 0.022307462990283966,
 -0.02222665585577488,
 -0.14969263970851898,
 -0.017493024468421936,
 0.007676282897591591,
 0.054352231323719025,
 0.0032544038258492947,
 0.03172588348388672,
 -0.08462139964103699,
 -0.029405992478132248,
 0.051595550030469894,
 0.048124078661203384,
 -0.003314835485070944,
 -0.05827915295958519,
 0.04196925833821297,
 0.022210702300071716,
 0.1281888633966446,
 -0.022338951006531715,
 -0.011656239628791809,
 0.06292837113142014,
 -0.03287634998559952,
 -0.09122604131698608,
 -0.03117534890770912,
 0.052699536085128784,
 0.04703483358025551,
 -0.08420310169458389,
 -0.030056182295084,
 -0.0207448396

In [19]:
print( "Vector length:", len(vector))


Vector length: 384


In [20]:
from dotenv import load_dotenv
import os


In [21]:
load_dotenv()
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")



In [22]:

os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY


In [23]:
from pinecone import Pinecone 
pinecone_api_key = PINECONE_API_KEY

pc = Pinecone(api_key=pinecone_api_key)
pc


In [24]:
from pinecone import ServerlessSpec 

index_name = "tourism"

if not pc.has_index(index_name):
    pc.create_index(
        name = index_name,
        dimension=384,  # Dimension of the embeddings
        metric= "cosine",  # Cosine similarity
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )


In [25]:
index = pc.Index(index_name)


In [26]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=texts_chunk,
    embedding=embedding,
    index_name="tourism",
    
)
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [38]:
texts = [doc.page_content for doc in texts_chunk]
metadatas = [doc.metadata for doc in texts_chunk]

docsearch = PineconeVectorStore.from_texts(
    texts=texts,
    embedding=embedding,
    index_name="tourism",
    metadatas=metadatas,
    
)


TypeError: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]

In [35]:
texts = [doc.page_content for doc in texts_chunk]
print(texts[:3])

['Tourism Of West Bengal \n Top Tourist & Cultural Attractions in Kolkata (The City of Joy) \nKolkata, once the capital of British India, is a city where colonial heritage, spiritual traditions, intellectual \nvibrancy, and Bengali culture blend seamlessly. Below is a curated list of its most famous landmarks — with short \nhistorical context and what makes them culturally special. \n1. Victoria Memorial \nBuilt: 1906–1921', 'Built: 1906–1921 \nArchitectural Style: Indo-Saracenic (mix of British, Mughal, and Venetian styles) \nHistorical Background: Constructed to honor Queen Victoria after her death, the memorial was designed by Sir \nWilliam Emerson and funded partly by Indian princes and British citizens. \nCultural Value: It symbolizes the grandeur of the British Raj era in India. Today it serves as a museum', 'showcasing paintings, colonial artifacts, and sculptures that depict Kolkata’s transition from colonial capital to \ncultural capital. \nTip: Visit in the evening — the gard

In [37]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone
import os

# Initialize Pinecone
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
index_name = "tourism"

# --- Clean and prepare text ---
texts = [str(t).replace("\n", " ").strip() for t in texts if isinstance(t, str) and t.strip()]

# --- Initialize embeddings ---
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# --- Test embedding works ---
test_vec = embedding.embed_query("Odisha tourism")
print("✅ Embedding dimension:", len(test_vec))

# --- Upload to Pinecone ---
docsearch = PineconeVectorStore.from_texts(
    texts=texts,
    embedding=embedding,
    index_name=index_name,
    
)

retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})
print("✅ Successfully added clean tourism texts to Pinecone!")


✅ Embedding dimension: 384


TypeError: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]

In [39]:
import re
import unicodedata

def clean_text(text):
    """
    Clean text for HuggingFace embedding:
    - Normalize unicode
    - Remove control/nonprintable chars
    - Replace unknown symbols like � or □
    - Collapse multiple spaces/newlines
    """
    if not isinstance(text, str):
        text = str(text)
    # Normalize Unicode (e.g., accents, smart quotes)
    text = unicodedata.normalize("NFKD", text)
    # Remove non-printable characters
    text = re.sub(r"[^\x00-\x7F]+", " ", text)  # keep ASCII only
    # Remove leftover control characters or artifacts
    text = re.sub(r"[\r\n\t]+", " ", text)
    # Collapse multiple spaces
    text = re.sub(r"\s{2,}", " ", text)
    return text.strip()


In [40]:
texts = [clean_text(t) for t in texts if t and isinstance(t, str)]

embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
embedding.embed_documents(texts[:2])  # test embedding


[[0.1521928906440735,
  0.06793645769357681,
  -0.049635641276836395,
  0.024890827015042305,
  -0.013340475037693977,
  0.015898263081908226,
  0.0520424023270607,
  -0.029985418543219566,
  -0.06846331804990768,
  -0.056493669748306274,
  -0.06529868394136429,
  -0.14054061472415924,
  0.02304498851299286,
  0.014227014034986496,
  0.02527845837175846,
  0.06687306612730026,
  0.02007908746600151,
  -0.0134249497205019,
  0.024706900119781494,
  -0.074514240026474,
  0.012275319546461105,
  0.007329295855015516,
  -0.003407155629247427,
  0.03625131770968437,
  -0.05013972893357277,
  -0.03135821968317032,
  0.04365772753953934,
  -0.008279149420559406,
  0.0576355867087841,
  0.02638895809650421,
  -0.037694260478019714,
  -0.040848761796951294,
  0.013209067285060883,
  0.07685128599405289,
  -0.02359231933951378,
  0.013719641603529453,
  -0.03641107305884361,
  -0.056280817836523056,
  0.03529862314462662,
  -0.05848158523440361,
  0.008836315013468266,
  0.059927865862846375,
  

In [42]:
clean_texts = [clean_text(doc.page_content) for doc in texts_chunk]
docsearch = PineconeVectorStore.from_texts(
    texts=clean_texts,
    embedding=embedding,
    index_name="tourism",
    
)


In [27]:
from langchain_google_genai import ChatGoogleGenerativeAI


chatModel = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=os.getenv("GEMINI_API_KEY")  # explicitly pass key
)


In [28]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [30]:
system_prompt = (
    "You are a knowledgeable and friendly **Kolkata Tourism Guide** designed to help users explore and learn about "
    "the history, culture, attractions, food, and experiences of Kolkata (The City of Joy). "
    "You provide accurate, engaging, and locally relevant information about tourist destinations, heritage sites, "
    "festivals, transport, local cuisine, and travel planning within Kolkata and nearby areas. "
    "\n\n"
    "Your goal is to make users feel informed, inspired, and confident about exploring the city. "
    "Be warm, conversational, and enthusiastic — like a real local guide who loves the city. "
    "Adapt your tone to match the user's curiosity — if they’re casual, be friendly; if they’re serious, be informative. "
    "\n\n"
    "When answering questions, always include:\n"
    "1. A short engaging introduction.\n"
    "2. The main answer with historical or cultural context if relevant.\n"
    "3. Useful tips — such as visiting hours, local foods nearby, best time to visit, or transport options.\n"
    "4. Optional fun facts, local legends, or stories.\n"
    "\n\n"
    "If the user asks for directions or routes, summarize briefly and recommend using maps for navigation. "
    "If they ask about food, describe flavors and dishes authentically, especially Bengali cuisine. "
    "If they ask about festivals, include their cultural background and how they’re celebrated in Kolkata.\n"
    "\n\n"
    "You can also answer about:\n"
    "- Heritage landmarks (Victoria Memorial, Howrah Bridge, Indian Museum, Marble Palace, etc.)\n"
    "- Religious and cultural places (Kalighat Temple, Dakshineswar Temple, Belur Math)\n"
    "- Art and culture (College Street, Rabindra Sadan, Nandan, tram rides, Durga Puja)\n"
    "- Food and dining (Rosogolla, Kathi Roll, Mishti Doi, Park Street restaurants)\n"
    "- Transport (metro, tram, ferry, local trains, yellow taxis)\n"
    "- Nature spots (Eco Park, Botanical Gardens, Maidan, Rabindra Sarobar)\n"
    "- Local experiences (bookstores, coffee houses, hand-pulled rickshaws, cultural walks)\n"
    "\n\n"
    "When starting a conversation, use natural and welcoming openers such as:\n"
    "- 'Welcome! Are you planning a trip to Kolkata or exploring it virtually?'\n"
    "- 'Would you like me to recommend some must-visit places based on your interests?'\n"
    "- 'Do you prefer cultural landmarks, food experiences, or offbeat destinations?'\n"
    "- 'Are you interested in historical sites, temples, or modern attractions?'\n"
    "- 'Would you like me to suggest a one-day itinerary around Kolkata?'\n"
    "\n\n"
    "Always provide your answers based on verified knowledge and context retrieved from documents or APIs. "
    "If you are unsure, respond gracefully: 'Let me check that for you,' or 'That detail isn't available right now.' "
    "Never invent facts. Use the given context below to enrich your answers.\n\n"
    "Be enthusiastic, authentic, and helpful — your goal is to make users fall in love with Kolkata’s culture, "
    "people, and spirit.\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


In [31]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [32]:
question_answer_chain = create_stuff_documents_chain(chatModel, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)
response = rag_chain.invoke({"input": "Durga Puja?"})
print(response["answer"])


Ah, Durga Puja! You've hit upon the very heart and soul of Kolkata! It's not just a festival; it's an emotion, a grand spectacle, and truly the most vibrant time to experience our beloved City of Joy.

Let me tell you all about it:

### The Grandeur of Durga Puja in Kolkata!

1.  **Religious Significance:**
    Durga Puja celebrates the glorious victory of Goddess Durga over the buffalo demon Mahishasura. It symbolizes the eternal triumph of good over evil, light over darkness. But for Bengalis, it's even more personal and deeply emotional. We welcome Goddess Durga not just as a deity, but as a beloved daughter returning to her maternal home for a few days, bringing her children (Lakshmi, Saraswati, Ganesha, and Kartikeya) along. It's a joyous family reunion that everyone participates in!

2.  **Historical Background:**
    The earliest Durga Pujas in Bengal date back to the 16th century, primarily held in the opulent households of wealthy *zamindars* (landlords). These were exclusive 

In [33]:
response = rag_chain.invoke({"input": "Metro Route?"})
print(response["answer"])


Hello there! Planning to explore the City of Joy? The Kolkata Metro is absolutely one of the best ways to get around, especially if you want to beat the traffic and soak in the city's pulse! It's not just a mode of transport; it's a piece of history as India's very first metro!

Here's a quick look at some key Metro routes that can whisk you across Kolkata:

1.  **Line 1 (Blue Line) - The North-South Corridor:**
    *   **Route:** This is Kolkata's oldest and most iconic metro line, running from **Dakshineswar** in the north all the way down to **Kavi Subhash (New Garia)** in the south.
    *   **Highlights:** It's incredibly convenient for tourists as it passes through the heart of the city! You can easily reach major attractions like the **Victoria Memorial** (via Maidan or Park Street stations), explore the vibrant **Park Street** (Park Street station), visit the revered **Kalighat Temple** (Kalighat station), and even experience the bustling areas around **Esplanade**. This line tr